### Using dataset "Equities" and the iterative nmf algorithm to analyse a large **FTSE100** dataset

In [17]:
#Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from NMF import nonNegMatrixFac
import numpy as np

**Convert the CSV dataset into a numpy matrix**

In [18]:
dataset = pd.read_excel('../Datasets/Equities.xlsx')
dataset

,Name,FTSE100,ADMIRAL GROUP,ANGLO AMERICAN,ANTOFAGASTA,ASHTEAD GROUP,ASSOCIATED BRIT.FOODS,ASTRAZENECA,AVIVA,BAE SYSTEMS,...,STANDARD LIFE ABERDEEN,TAYLOR WIMPEY,TESCO,UNILEVER (UK),UNITED UTILITIES GROUP,VODAFONE GROUP,WHITBREAD,WOOD GROUP (JOHN),WPP,3I GROUP
0,2011-08-18,5092.23,1492.0,2299.0,1170.0,103.1,1035,2743.5,322.0,253.8,...,195.28,29.00,371.55,2025.0,580.0,166.50,1445,526.5,601.0,204.3
1,2011-08-19,5040.76,1495.0,2238.5,1183.0,101.0,1020,2731.5,315.3,253.5,...,194.77,28.71,370.25,2007.0,571.5,163.93,1429,507.5,590.0,202.0
2,2011-08-22,5095.30,1521.0,2249.5,1212.0,100.6,1041,2822.0,318.0,258.1,...,198.64,28.90,377.70,2038.0,580.5,167.06,1454,518.5,578.5,203.4
3,2011-08-23,5129.42,1535.0,2234.0,1206.0,99.4,1049,2832.5,318.8,258.8,...,197.93,29.10,377.20,2042.0,579.5,168.75,1445,542.5,580.0,200.7
4,2011-08-24,5205.85,1353.0,2287.0,1203.0,101.0,1049,2873.5,327.4,266.9,...,203.64,29.52,377.30,2043.0,586.5,172.18,1441,554.5,623.0,207.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,2019-02-22,7178.60,2185.0,2057.0,946.0,2057.0,2313,6159.0,425.7,470.8,...,235.60,167.55,222.80,4190.0,852.8,141.80,4894,532.4,845.0,953.0
1962,2019-02-25,7183.74,2187.0,2039.0,966.2,2058.0,2273,6290.0,428.8,475.9,...,239.55,165.35,223.40,4156.5,850.8,139.60,4901,528.6,855.0,955.8
1963,2019-02-26,7151.12,2182.0,2051.5,968.8,2034.0,2319,6271.0,430.0,472.6,...,242.50,170.95,228.40,4117.0,850.0,136.60,4910,545.0,842.8,958.0
1964,2019-02-27,7107.20,2169.0,2030.0,964.4,1995.0,2255,6283.0,423.4,469.7,...,242.20,177.00,229.10,3988.0,839.2,133.60,4854,538.8,832.6,935.4


In [19]:
dataset = dataset.drop(['FTSE100','Name'], axis=1)
dataset

,ADMIRAL GROUP,ANGLO AMERICAN,ANTOFAGASTA,ASHTEAD GROUP,ASSOCIATED BRIT.FOODS,ASTRAZENECA,AVIVA,BAE SYSTEMS,BARCLAYS,BARRATT DEVELOPMENTS,...,STANDARD LIFE ABERDEEN,TAYLOR WIMPEY,TESCO,UNILEVER (UK),UNITED UTILITIES GROUP,VODAFONE GROUP,WHITBREAD,WOOD GROUP (JOHN),WPP,3I GROUP
0,1492.0,2299.0,1170.0,103.1,1035,2743.5,322.0,253.8,142.26,70.85,...,195.28,29.00,371.55,2025.0,580.0,166.50,1445,526.5,601.0,204.3
1,1495.0,2238.5,1183.0,101.0,1020,2731.5,315.3,253.5,139.02,67.50,...,194.77,28.71,370.25,2007.0,571.5,163.93,1429,507.5,590.0,202.0
2,1521.0,2249.5,1212.0,100.6,1041,2822.0,318.0,258.1,135.10,68.60,...,198.64,28.90,377.70,2038.0,580.5,167.06,1454,518.5,578.5,203.4
3,1535.0,2234.0,1206.0,99.4,1049,2832.5,318.8,258.8,134.36,71.20,...,197.93,29.10,377.20,2042.0,579.5,168.75,1445,542.5,580.0,200.7
4,1353.0,2287.0,1203.0,101.0,1049,2873.5,327.4,266.9,138.19,71.05,...,203.64,29.52,377.30,2043.0,586.5,172.18,1441,554.5,623.0,207.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,2185.0,2057.0,946.0,2057.0,2313,6159.0,425.7,470.8,156.14,575.20,...,235.60,167.55,222.80,4190.0,852.8,141.80,4894,532.4,845.0,953.0
1962,2187.0,2039.0,966.2,2058.0,2273,6290.0,428.8,475.9,159.22,573.20,...,239.55,165.35,223.40,4156.5,850.8,139.60,4901,528.6,855.0,955.8
1963,2182.0,2051.5,968.8,2034.0,2319,6271.0,430.0,472.6,164.90,589.20,...,242.50,170.95,228.40,4117.0,850.0,136.60,4910,545.0,842.8,958.0
1964,2169.0,2030.0,964.4,1995.0,2255,6283.0,423.4,469.7,166.90,599.20,...,242.20,177.00,229.10,3988.0,839.2,133.60,4854,538.8,832.6,935.4


In [20]:
# Apply NMF to reduce the dimentionality
 
dataset = dataset.to_numpy()

m, n = dataset.shape
print(dataset.shape)
r = 10

W = np.random.rand(m,r)
H = np.random.rand(r,n)

V_norm = np.zeros((m,n))

for i in range(n):
    V_norm[:,i] = dataset[:,i] / np.max(dataset[:,i])

mnf = nonNegMatrixFac(W, H, 30000)
mnf.fit(V_norm)

(1966, 95)


(array([[2.08971375e-01, 1.53636757e-01, 2.83896240e-02, ...,
         1.25169580e-01, 3.21010152e-01, 1.36421481e-02],
        [1.57684709e-01, 1.62926992e-01, 2.78260759e-21, ...,
         8.14214793e-02, 3.51613473e-01, 4.22679121e-10],
        [1.55563034e-01, 1.39795211e-01, 3.42016569e-63, ...,
         5.45409271e-02, 3.61451673e-01, 8.23304667e-07],
        ...,
        [3.32175121e-02, 3.77155779e-19, 1.58329090e-01, ...,
         1.16514013e+00, 5.74454647e-01, 2.73684041e+00],
        [2.86294284e-02, 8.03203855e-12, 1.58237633e-01, ...,
         1.13240198e+00, 5.90378780e-01, 2.71211911e+00],
        [2.68271915e-03, 2.16204251e-28, 1.44196504e-01, ...,
         1.16192130e+00, 6.26309580e-01, 2.73232316e+00]]),
 array([[1.26959692e-001, 1.77023777e-001, 6.51309579e-002,
         5.54849649e-016, 1.49878794e-001, 1.00128748e-001,
         1.78197980e-001, 9.49308042e-002, 4.60273910e-002,
         3.15002282e-003, 6.89239355e-002, 1.96497779e-001,
         2.03139446e-001,

In [21]:
mnf.W.shape

(1966, 10)